# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()

key = "Heart-Failure"

if key in ws.datasets.keys(): 
        dataset = ws.datasets[key] 

# choose a name for experiment
experiment_name = 'Heart-Fail-Pred'

experiment=Experiment(ws, experiment_name)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [ ]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'AUC_weighted',
    "n_cross_validations": 5,
    "featurization": 'auto'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [ ]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

In [ ]:
#TODO: Save the best model
joblib.dump(value=fitted_model, filename="best-trained-model.pkl")

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
from azureml.core.model import Model
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Workspace, Environment


model = Model.register(model_path="best-trained-model.pkl",
                       model_name="best-trained-model",
                       workspace = ws)



service_name = 'my-custom-env-service'

env = Environment.get(workspace=ws, name="AzureML-Minimal")

inference_config = InferenceConfig(entry_script='score.py', environment=env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                          name=service_name,
                          models=[model],
                          inference_config=inference_config,
                          deployment_config=aci_config
                       
service.wait_for_deployment(show_output=True)

# enable application insight
service.update(enable_app_insights=True)

print(service.state)
print("scoring URI: " + service.scoring_uri)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
import requests
import json

scoring_uri = service.scoring_uri
headers = {'Content-Type':'application/json'}

test_data = json.dumps({'age': 75, 'anaemia': 0, 'creatinine_phosphokinase': 582, 'diabetes': 0, 'ejection_fraction':20, 'high_blood_pressure': 1, 'platelets': 265000, 'serum_creatinine': 1.9, 'serum_sodium': 130, 'sex':1, 'smoking':0 })

response = requests.post(scoring_uri, data=test_data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
logs = service.get_logs()

for line in logs.split('\n'):
    print(line)